In [9]:
pip install git+https://github.com/facebookresearch/fastText.git

  Cloning https://github.com/facebookresearch/fastText.git to c:\users\user\appdata\local\temp\pip-req-build-_e1_tl8m
  Resolved https://github.com/facebookresearch/fastText.git to commit 1142dc4c4ecbc19cc16eee5cdd28472e689267e6
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build fasttext
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fastText.git 'C:\Users\USER\AppData\Local\Temp\pip-req-build-_e1_tl8m'
  error: subprocess-exited-with-error
  
  × Building wheel for fasttext (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [45 lines of output]
      C:\Users\USER\AppData\Local\Temp\pip-build-env-_holie66\overlay\Lib\site-packages\setuptools\dist.py:599: SetuptoolsDeprecationWarning: Invalid dash-separated key 'description-file' in 'metadata' (setup.cfg), please use the underscore name 'description_file' instead.
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
              (Affected: fasttext).
      
              By 2026-Mar-03, you need to update your project and remove depre

In [10]:
pip install fasttext-wheel

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd

df = pd.read_csv("datasets/cleanedDataSecondModel_merged.csv")

# Convert to fastText format
def save_fasttext_format(df, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for text, label in zip(df['quote'], df['category']):
            line = f"__label__{label} {text.strip()}\n"
            f.write(line)

# Save to training and test files
save_fasttext_format(df.sample(frac=0.8, random_state=42), "datasets/FTtrain.txt")
save_fasttext_format(df.drop(df.sample(frac=0.8, random_state=42).index), "datasets/FTtest.txt")


In [12]:
import fasttext

# Train the classifier
model = fasttext.train_supervised(input="datasets/FTtrain.txt", epoch=5, lr=1, wordNgrams=4, verbose=2)

# Save the model (optional)
model.save_model("models/fasttext_model.bin")


In [13]:
# Evaluate on test set
model.test("datasets/FTtest.txt")


(200, 0.09, 0.09)

In [14]:
quote = "dont give up"
label, prob = model.predict(quote)

print("Predicted Category:", label[0].replace("__label__", ""))
print("Confidence:", prob[0])


Predicted Category: other
Confidence: 0.06801826506853104


In [15]:
from sklearn.metrics import classification_report, confusion_matrix
test_texts = []
true_labels = []

with open("datasets/FTtest.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split(" ", 1)
        if len(parts) == 2:
            label, text = parts
            test_texts.append(text)
            true_labels.append(label.replace("__label__", ""))


predicted_labels = []

for text in test_texts:
    label, _ = model.predict(text)
    predicted_labels.append(label[0].replace("__label__", ""))


print(classification_report(true_labels, predicted_labels))



                precision    recall  f1-score   support

      ambition       0.00      0.00      0.00         2
  appreciating       0.00      0.00      0.00         5
  appreciation       0.00      0.00      0.00         1
       believe       0.00      0.00      0.00         4
        caring       0.00      0.00      0.00         1
     character       0.00      0.00      0.00         1
         cheer       0.00      0.00      0.00         3
    commitment       0.00      0.00      0.00         3
        common       0.00      0.00      0.00         2
    compassion       0.00      0.00      0.00         5
   compliments       0.00      0.00      0.00         2
    confidence       0.00      0.00      0.00         7
       courage       0.00      0.00      0.00         4
      courtesy       0.00      0.00      0.00         2
    creativity       0.00      0.00      0.00         5
    dedication       0.00      0.00      0.00         1
         drive       0.00      0.00      0.00  

c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le